# LawIT: Trata Brujula

## Part 1: Data Cleaning and Merging

In [1]:
# Importing libraries

import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#Calling all inputs from a file called datasets

files = glob.glob('datasets/*.csv')
for x in files:
    print(x)

datasets/tasa-denuncia-delitos.csv
datasets/violencia-sicologica.csv
datasets/educacion15amas.csv
datasets/analfabetimos.csv
datasets/tasadesempleo.xls.csv
datasets/canon.csv
datasets/empleoinformal.csv
datasets/poblacionconsegurosalud.csv
datasets/Casos intervenidos  por el delito de trata de personas.csv
datasets/acceso-agua.csv
datasets/Al-menos-1-necesidad-no-satis.csv
datasets/pob-total-estimada.csv
datasets/Incidencia-pobreza.csv
datasets/violencia-fisica.csv


In [3]:
# merging all files in a single dataframe

dptos = ['amazonas', 'ancash', 'apurimac', 'arequipa', 'ayacucho', 'cajamarca', 'callao',
 'cusco', 'huancavelica', 'huanuco', 'ica', 'junin', 'la libertad', 'lambayeque',
 'lima', 'loreto', 'madre de dios' ,'moquegua', 'pasco', 'piura', 'puno',
 'san martin', 'tacna', 'tumbes', 'ucayali']

df_list = []
for file in files:
    print("reading: ", file)
    try:        
        temp_df = pd.read_csv(file, sep=",")                               # read the df when sep is ,
        temp_df.columns = [x.lower() for x in temp_df.columns]             # modify column names
        temp_df["departamento"] = temp_df["departamento"].str.lower()      # convert to lower case
        temp_df["indicador"] = temp_df["indicador"].str.lower()            # convert to lower case
        
        if(set(temp_df["departamento"].unique()) != set(dptos)):        # check is the department names conform
            print(set(temp_df["departamento"].unique()) - set(dptos))
            
        df_list.append(temp_df)
    
    except:
        temp_df = pd.read_csv(file, sep=";")                               # read the df when sep ;
        temp_df.columns = [x.lower() for x in temp_df.columns]             # modify column names
        temp_df["departamento"] = temp_df["departamento"].str.lower()      # convert to lower case
        temp_df["indicador"] = temp_df["indicador"].str.lower()            # convert to lower case
        
        if(set(temp_df["departamento"].unique()) != set(dptos)):
            print(temp_df["departamento"].unique())
            
        df_list.append(temp_df)
        

reading:  datasets/tasa-denuncia-delitos.csv
reading:  datasets/violencia-sicologica.csv
reading:  datasets/educacion15amas.csv
reading:  datasets/analfabetimos.csv
reading:  datasets/tasadesempleo.xls.csv
reading:  datasets/canon.csv
reading:  datasets/empleoinformal.csv
reading:  datasets/poblacionconsegurosalud.csv
reading:  datasets/Casos intervenidos  por el delito de trata de personas.csv
reading:  datasets/acceso-agua.csv
reading:  datasets/Al-menos-1-necesidad-no-satis.csv
reading:  datasets/pob-total-estimada.csv
reading:  datasets/Incidencia-pobreza.csv
reading:  datasets/violencia-fisica.csv


In [4]:
comp_df = pd.concat(df_list)                        # join all data frames
comp_df = comp_df.drop("2019", axis=1)              # eliminate 2019 column

assert(len(comp_df["departamento"].unique())==25)  # check there are 25 departments

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
# create long Dataframe (group years in a single variable)

long_df = pd.melt(comp_df,
                    id_vars=["departamento", "indicador"],
                   var_name="año",
                   value_name="valor")

long_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 4 columns):
departamento    1050 non-null object
indicador       1050 non-null object
año             1050 non-null object
valor           1050 non-null float64
dtypes: float64(1), object(3)
memory usage: 32.9+ KB


In [14]:
# create wide data frame (convert "indicador" in multiple columns)

wide_df = long_df.set_index(['departamento', 'año', 'indicador'])
wide_df = wide_df.unstack(level=-1)
wide_df.columns = wide_df.columns.droplevel()
wide_df = wide_df.reset_index(level=['departamento', 'año'])
wide_df.columns.name = None
wide_df["año"] = wide_df["año"].astype("int")

wide_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 16 columns):
departamento                                                   75 non-null object
año                                                            75 non-null int64
canon minero                                                   75 non-null float64
casos intervenidos  por el delito de trata de personas         75 non-null float64
empleo informal miles                                          75 non-null float64
incidencia de pobreza                                          75 non-null float64
indice analfabetismo                                           75 non-null float64
indice de acceso a agua                                        75 non-null float64
pob-total-estimada                                             75 non-null float64
poblacion con al menos una necesidad basica insatisfecha       75 non-null float64
porcentaje de la población afiliada a algún seguro de salud    75 non-

In [16]:
wide_df.sample(10)

,departamento,año,canon minero,casos intervenidos por el delito de trata de personas,empleo informal miles,incidencia de pobreza,indice analfabetismo,indice de acceso a agua,pob-total-estimada,poblacion con al menos una necesidad basica insatisfecha,porcentaje de la población afiliada a algún seguro de salud,prom eduacion de 15 anos a mas,tasa de desempleo,tasa denuncia de delitos,violencia fisica,violencia sicologica
4,ancash,2017,494475.0,10.0,493.0,24.60,10.0,92.3,1140486.0,14.934690,80.6,9.5,2.9,101.7,33.1,66.4
29,huanuco,2018,12422.0,18.0,402.8,34.55,12.9,75.2,757467.0,23.670195,82.9,8.7,1.9,70.4,26.6,64.5
9,arequipa,2016,21985.0,45.0,442.8,10.80,4.4,93.4,1357444.0,11.111693,63.2,10.9,4.9,151.1,38.3,70.1
49,madre de dios,2017,87391.0,143.0,62.4,3.45,4.3,88.6,155027.0,22.840103,67.7,9.9,1.5,167.1,34.9,59.5
25,huancavelica,2017,980.0,37.0,241.3,35.05,13.8,84.1,381277.0,20.910725,93.5,8.7,2.4,29.5,35.5,69.1
60,puno,2016,87175.0,98.0,657.6,34.25,9.2,67.6,1231778.0,26.664152,70.1,9.6,3.1,33.2,42.2,76.0
16,cajamarca,2017,213291.0,8.0,784.3,47.55,11.5,81.8,1427598.0,21.394125,84.5,8.2,1.9,61.5,25.9,57.2
73,ucayali,2017,0.0,17.0,216.5,13.35,4.5,71.6,542847.0,42.293848,75.2,9.4,2.7,138.6,24.4,44.3
74,ucayali,2018,0.0,19.0,215.6,12.55,4.5,76.0,558767.0,40.552625,74.2,9.7,2.7,166.6,25.2,45.6
8,apurimac,2018,11708.0,4.0,231.4,34.55,14.0,95.4,427323.0,11.683063,92.8,9.1,1.9,97.9,44.3,79.5


In [17]:
# save the final dataframe
wide_df.to_csv("data.csv")